In [2]:
from __future__ import print_function, division, absolute_import
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
# set_session(tf.Session(config=config))
# from clr_callback import CyclicLR
# import dill
from BalancedDannAudioDataGenerator import BalancedAudioDataGenerator
import os,time
from scipy.io import loadmat
import numpy as np
np.random.seed(1)
from tensorflow import set_random_seed
set_random_seed(1)
import math
import pandas as pd
import tables,h5py
from datetime import datetime
import argparse
from keras.callbacks import TensorBoard, Callback, ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, CSVLogger
from keras import backend as K
from keras.utils import plot_model
from Heartnet import heartnet,getAttentionModel
from utils import log_macc, results_log
from dataLoader import reshape_folds
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns
import Evaluator
import dataLoader
from custom_layers import Attention
from sklearn.model_selection import train_test_split
sns.set()
import tensorflow as tf
from collections import Counter

In [4]:
from custom_layers import Conv1D_zerophase_linear, Conv1D_linearphase, Conv1D_zerophase,\
    DCT1D, Conv1D_gammatone, Conv1D_linearphaseType, Attention
from keras.layers import Input, Conv1D, MaxPooling1D, Dense, Dropout, Flatten, Activation, AveragePooling1D, Add
from keras import initializers
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.layers.core import Lambda
from keras.regularizers import l2
from keras.constraints import max_norm
from keras.optimizers import Adam, SGD # Nadam, Adamax
import numpy as np
import tables,h5py
from Gradient_Reverse_Layer import GradientReversal
from ResultAnalyser import Result
from utils import Confused_Crossentropy
from keras.utils import plot_model
from keras import backend as K


In [9]:
def branch(input_tensor,num_filt,kernel_size,random_seed,padding,bias,maxnorm,l2_reg,
           eps,bn_momentum,activation_function,dropout_rate,subsam,trainable):

    num_filt1, num_filt2 = num_filt
    t = Conv1D(num_filt1, kernel_size=kernel_size,
                kernel_initializer=initializers.he_normal(seed=random_seed),
                padding=padding,
                use_bias=bias,
                kernel_constraint=max_norm(maxnorm),
                trainable=trainable,
                kernel_regularizer=l2(l2_reg))(input_tensor)
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    t = Activation(activation_function)(t)
    t = Dropout(rate=dropout_rate, seed=random_seed)(t)
    t = Conv1D(16, kernel_size=kernel_size,
               kernel_initializer=initializers.he_normal(seed=random_seed),
               padding=padding,
               use_bias=bias,
               trainable=trainable,
               kernel_constraint=max_norm(maxnorm),
               kernel_regularizer=l2(l2_reg))(t)
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    t = Activation(activation_function)(t)
    t = Dropout(rate=dropout_rate, seed=random_seed)(t)
    # t = Flatten()(t)
    return t
def zeropad(x):
    y = K.zeros_like(x)
    return K.concatenate([x, y], axis=2)

def zeropad_output_shape(input_shape):
    shape = list(input_shape)
    assert len(shape) == 3
    shape[2] *= 2
    return tuple(shape)

In [5]:
num_filt = (8, 4)
num_dense = 20
lr = 0.01
bn_momentum = 0.99
eps = 1.1e-5
bias = False
l2_reg = 0.04864911065093751
l2_reg_dense = 0.
kernel_size = 5
maxnorm = 10000.
dropout_rate = 0.5
dropout_rate_dense = 0.
padding = 'valid'
activation_function = 'relu'
subsam = 2
FIR_train= True
trainable = True
hp_lambda = np.float32(0)
lr_decay =0.0001132885
random_seed = 1
num_class =2
num_class_domain = 18
tipe= 1
decision = 'majority' 
channels = '0101'

In [10]:
def res_block(input_tensor,num_filt,kernel_size,stride,padding,random_seed,bias,maxnorm,l2_reg,
           eps,bn_momentum,activation_function,dropout_rate,subsam,trainable,cat=True):

    t = Conv1D(num_filt, kernel_size=kernel_size,
                kernel_initializer=initializers.he_normal(seed=random_seed),
                padding=padding,
                strides=stride,
                use_bias=bias,
                kernel_constraint=max_norm(maxnorm),
                trainable=trainable,
                kernel_regularizer=l2(l2_reg))(input_tensor)
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    t = Activation(activation_function)(t)
    t = Dropout(rate=dropout_rate, seed=random_seed)(t)
    t = Conv1D(num_filt, kernel_size=kernel_size,
                kernel_initializer=initializers.he_normal(seed=random_seed),
                padding=padding,
                strides=1,
                use_bias=bias,
                kernel_constraint=max_norm(maxnorm),
                trainable=trainable,
                kernel_regularizer=l2(l2_reg))(t)
    t = BatchNormalization(epsilon=eps, momentum=bn_momentum, axis=-1)(t)
    t = Activation(activation_function)(t)
    t = Dropout(rate=dropout_rate, seed=random_seed)(t)
    
    p = MaxPooling1D(pool_size=stride)(input_tensor)
    if(stride>1):
        if(cat):
            p = Lambda(zeropad, output_shape=zeropad_output_shape)(p)
    
    t = Add()([t,p])
    return t

In [15]:
def heartnet(load_path,activation_function='relu', bn_momentum=0.99, bias=False, dropout_rate=0.5, dropout_rate_dense=0.0,
             eps=1.1e-5, kernel_size=5, l2_reg=0.0, l2_reg_dense=0.0,lr=0.0012843784, lr_decay=0.0001132885, maxnorm=10000.,
             padding='valid', random_seed=1, subsam=2, num_filt=(8, 4), num_dense=20,FIR_train=False,trainable=True,type=1,
             num_class=2, num_class_domain=1,hp_lambda=0,batch_size=1024,optim='SGD',segments = channels):
    
    #num_dense = 20 default 
    input = Input(shape=(2500, 1))

    input2 = Lambda(lambda x:x[:,200:1000,:])(input)
    input4 = Lambda(lambda x:x[:,1200:2900,:])(input)
    X = []
    for i in range(2):
        if(i==0):
            xx = branch(input2,num_filt,kernel_size,random_seed,padding,bias,maxnorm,l2_reg,
                   eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)
        else:
            xx = branch(input4,num_filt,kernel_size,random_seed,padding,bias,maxnorm,l2_reg,
                   eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)
        xx = branch(input,num_filt,kernel_size,random_seed,padding,bias,maxnorm,l2_reg,
               eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)
        xx = res_block(xx,32,kernel_size,2,'same',random_seed,bias,maxnorm,l2_reg,
               eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)
        xx = res_block(xx,64,kernel_size,2,'same',random_seed,bias,maxnorm,l2_reg,
               eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)

        xx = MaxPooling1D(pool_size=2)(xx)

    #     xx = res_block(xx,128,kernel_size,2,'same',random_seed,bias,maxnorm,l2_reg,
    #            eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)
    #     xx = res_block(xx,128,kernel_size,1,'same',random_seed,bias,maxnorm,l2_reg,
    #            eps,bn_momentum,activation_function,dropout_rate,subsam,trainable)

    #     xx = res_block(xx,128,kernel_size,2,'same',random_seed,bias,maxnorm,l2_reg,
    #            eps,bn_momentum,activation_function,dropout_rate,subsam,trainable,cat=False)
    #     xx = res_block(xx,128,kernel_size,2,'same',random_seed,bias,maxnorm,l2_reg,
    #            eps,bn_momentum,activation_function,dropout_rate,subsam,trainable,cat=False)

        xx = Conv1D(64, kernel_size=kernel_size,
                    kernel_initializer=initializers.he_normal(seed=random_seed),
                    padding=padding,
                    strides=2,
                    use_bias=bias,
                    kernel_constraint=max_norm(maxnorm),
                    trainable=trainable,
                    kernel_regularizer=l2(l2_reg))(xx)
        xx = MaxPooling1D(pool_size=2)(xx)
        X.append(xx)
    merged = Concatenate(axis=-1)([X[0], X[1]])
    merged = Flatten()(merged)
    merged = Dropout(rate=dropout_rate, seed=random_seed)(merged)
    
    
    dann_in = GradientReversal(hp_lambda=hp_lambda,name='grl')(merged)
    dsc = Dense(80,
                   activation=activation_function,
                   kernel_initializer=initializers.he_normal(seed=random_seed),
                   use_bias=bias,
                   kernel_constraint=max_norm(maxnorm),
                   kernel_regularizer=l2(l2_reg_dense),
                   name = 'domain_dense')(dann_in)   
    dsc = Dense(num_class_domain, activation='softmax', name = "domain")(dsc)          
    merged = Dense(50,
                   activation=activation_function,
                   kernel_initializer=initializers.he_normal(seed=random_seed),
                   use_bias=bias,
                   kernel_constraint=max_norm(maxnorm),
                   kernel_regularizer=l2(l2_reg_dense),
                   name = 'class_dense')(merged)
    merged = Dense(num_class, activation='softmax', name="class")(merged)
    
    model = Model(inputs=input, outputs=[merged,dsc])
    
    if load_path:
        model.load_weights(filepath=load_path, by_name=False)
    
    #if load_path:  # If path for loading model was specified
    #model.load_weights(filepath='../../models_dbt_dann/fold_a_gt 2019-09-09 16:53:52.063276/weights.0041-0.6907.hdf5', by_name=True)
    # models/fold_a_gt 2019-09-04 17:36:52.860817/weights.0200-0.7135.hdf5
    
    if optim=='Adam':
        opt = Adam(lr=lr, decay=lr_decay)
    else:  
        opt = SGD(lr=lr,decay=lr_decay)
    if(num_class_domain>1):
        domain_loss_function = 'categorical_crossentropy'
    else:
        domain_loss_function = 'binary_crossentropy'
    model.compile(optimizer=opt, loss={'class':'categorical_crossentropy','domain':domain_loss_function}, metrics=['accuracy'])
    #model.compile(optimizer=opt, loss=['categorical_crossentropy','categorical_crossentropy'], metrics=['accuracy'])
    return model

In [16]:
model = heartnet(None,activation_function, bn_momentum, bias, dropout_rate, dropout_rate_dense,
                             eps, kernel_size, l2_reg, l2_reg_dense, lr, lr_decay, maxnorm,
                             padding, random_seed, subsam, num_filt, num_dense, FIR_train, trainable, tipe,
                             num_class=num_class,num_class_domain=9,hp_lambda=hp_lambda,segments=channels)

In [5]:
plot_model(model, to_file='smallNetS1S2.png', show_shapes=True)

NameError: name 'model' is not defined

In [23]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 2500, 1)      0                                            
__________________________________________________________________________________________________
conv1d_57 (Conv1D)              (None, 2496, 8)      40          input_4[0][0]                    
__________________________________________________________________________________________________
conv1d_66 (Conv1D)              (None, 2496, 8)      40          input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_51 (BatchNo (None, 2496, 8)      32          conv1d_57[0][0]                  
__________________________________________________________________________________________________
batch_norm

In [8]:
from HeartSegNet import heartnet

In [9]:
model = heartnet(None,activation_function, bn_momentum, bias, dropout_rate, dropout_rate_dense,
                             eps, kernel_size, l2_reg, l2_reg_dense, lr, lr_decay, maxnorm,
                             padding, random_seed, subsam, num_filt, num_dense, FIR_train, trainable, tipe,
                             num_class=num_class,num_class_domain=9,hp_lambda=hp_lambda,segments=channels)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [34]:
del heartnet

In [37]:
type(heartnet)

function